In [47]:
import pandas as pd 
import numpy as np
def print_full_rows(x):
    pd.set_option('display.max_rows', len(x))
    return x
    pd.reset_option('display.max_rows')
    
def print_full_columns(x):
    pd.set_option('display.max_columns', len(x.columns))
    return x
    pd.reset_option('display.max_rows')

### Concatenating all 2019 Data 

In [48]:
loan_df_2019 = pd.read_csv('loandata_2019.csv')

### Preprocessing the data

1. Dropping the values with more than 50% Missing values
2. Remove features that are not identifiable before issuing a loan
3. Convert Categorical variables / Log scale 
4. 

In [49]:
df = loan_df_2019.copy()

### Only using two binary state of the loan status

In [50]:
df.columns

Index(['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade',
       ...
       'orig_projected_additional_accrued_interest',
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'debt_settlement_flag', 'debt_settlement_flag_date',
       'settlement_status', 'settlement_date', 'settlement_amount',
       'settlement_percentage', 'settlement_term'],
      dtype='object', length=151)

In [51]:
df = df[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]

In [52]:
def null_values(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [53]:
null_values(df)

Dataframe has 151 columns.
There are 61 columns that have missing values.


,Missing Values,% of Total Values
member_id,118985,100.0
next_pymnt_d,118985,100.0
desc,118985,100.0
settlement_percentage,118125,99.3
settlement_amount,118125,99.3
...,...,...
last_pymnt_d,863,0.7
dti,296,0.2
revol_util,185,0.2
all_util,32,0.0


In [54]:
missing_frac = df.isnull().mean()
drop_list = sorted(missing_frac[missing_frac >= 0.50].index)
print(drop_list)

['annual_inc_joint', 'debt_settlement_flag_date', 'deferral_term', 'desc', 'dti_joint', 'hardship_amount', 'hardship_dpd', 'hardship_end_date', 'hardship_last_payment_amount', 'hardship_length', 'hardship_loan_status', 'hardship_payoff_balance_amount', 'hardship_reason', 'hardship_start_date', 'hardship_status', 'hardship_type', 'member_id', 'mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_last_record', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq', 'next_pymnt_d', 'orig_projected_additional_accrued_interest', 'payment_plan_start_date', 'revol_bal_joint', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_earliest_cr_line', 'sec_app_fico_range_high', 'sec_app_fico_range_low', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_mths_since_last_major_derog', 'sec_app_num_rev_accts', 'sec_app_open_acc', 'sec_app_open_act_il', 'sec_app_revol_util', 'settlement_amount', 'settlement_date', 'settlement_percentage', 'settlem

In [55]:
def drop_cols(cols):
    df.drop(labels=cols, axis=1, inplace=True)

In [56]:
drop_cols(drop_list)

In [57]:
df.shape

(118985, 107)

In [58]:
drop_list2 = ['acc_now_delinq', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'debt_settlement_flag', 'delinq_2yrs', 'delinq_amnt', 'funded_amnt', 'funded_amnt_inv', 'hardship_flag', 'inq_last_6mths', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'last_pymnt_amnt', 'last_pymnt_d', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m',  'out_prncp', 'out_prncp_inv', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pymnt_plan', 'recoveries', 'tax_liens', 'tot_coll_amt', 'tot_cur_bal', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'total_rev_hi_lim']

drop_cols(drop_list2)

In [59]:
drop_cols(['id','grade','emp_title','title','zip_code','policy_code', 'url','open_acc_6m', 'open_act_il', 'open_il_12m','open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util','open_rv_12m','open_rv_24m','open_rv_24m','max_bal_bc','all_util','inq_fi','total_cu_tl','inq_last_12m'])

In [60]:
# Fix the term feature
df['term'].sample(5)
df['term'] = df['term'].apply(lambda s: np.int8(s.split()[0]))

In [61]:
# Fix the emp_length feature (convert to integer)
df['emp_length'].replace('10+ years', '10 years', inplace=True)
df['emp_length'].replace('< 1 year', '0 years', inplace=True)
df['emp_length'].value_counts(dropna=False).sort_index()
df.emp_length.map( lambda x: str(x).split()[0]).value_counts(dropna=True).sort_index()
df['emp_length'] = df.emp_length.map(lambda x: float(str(x).split()[0]))
df['emp_length'].sample(5)

48977     10.0
202107     1.0
133872     0.0
433519    10.0
416341     NaN
Name: emp_length, dtype: float64

In [62]:
# Fix the int_rate feature
df['int_rate'].replace(regex=True, inplace=True, to_replace=r'%', value=r'')
df['int_rate'] = df['int_rate'].apply(pd.to_numeric, errors='coerce')

In [63]:
# Fix the annual income variable, use log-transform since there is a large range of variation, we log-transform the values.
df['annual_inc'] = df['annual_inc'].apply(lambda x:np.log10(x+1))

In [64]:
# fix earliest_cr_line to datetime variable
from datetime import datetime

df.earliest_cr_line = pd.to_datetime(df.earliest_cr_line)
dttoday = datetime.now().strftime('%Y-%m-%d')
df.earliest_cr_line = df.earliest_cr_line.apply(lambda x:(np.timedelta64((x - pd.Timestamp(dttoday)),'D').astype(int))/-365)
df.earliest_cr_line.shape

(118985,)

In [65]:
# create new variable fico_score, which is the mean value of fico_low and fico_high
df['fico_score'] = (df['fico_range_low'] + df['fico_range_high'])/2.
drop_cols(['fico_range_high','fico_range_low'])

In [66]:
# Change the bin width of pub_rec
df.pub_rec = df.pub_rec.map(lambda x: 3 if x >2.0 else x)

In [67]:
# Fix revol_bal
df['revol_bal'] = df['revol_bal'].apply(lambda x:np.log10(x+1))

In [68]:
# Fix revol_util
df['revol_util'] = df['revol_util'].replace(regex=True, inplace=False, to_replace=r'%', value=r'')

In [69]:
df['revol_util'] = df['revol_util'].apply(pd.to_numeric, errors='coerce')

In [70]:
# Change target variable to be a binary variable. Fully Paid = 0, Charged Off = 1
df['Charged_Off'] = df['loan_status'].apply(lambda s: np.float(s == 'Charged Off'))
drop_cols('loan_status')
df.shape

(118985, 28)

In [71]:
# drop columns that are not linearly correlated with the target variable. 
drop_cols(['installment', 'mo_sin_old_rev_tl_op','total_acc','pub_rec_bankruptcies', 'issue_d'])
df.shape

(118985, 23)

In [72]:
dummy_list =['sub_grade','home_ownership','verification_status','purpose','addr_state','initial_list_status','application_type']
df[dummy_list].isnull().any()
df = pd.get_dummies(df, columns=dummy_list, drop_first=True)

In [73]:
df['Charged_Off'].value_counts()[0]

97048

In [74]:
fully_paid = (df['Charged_Off'].value_counts()[0])/len(df)
charged_off = (df['Charged_Off'].value_counts()[1])/len(df)
(fully_paid, charged_off)


(0.8156322225490608, 0.18436777745093919)

# Modeling Begin

In [75]:
X = df.drop('Charged_Off', axis =1)
y = df['Charged_Off']

In [76]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)



In [77]:
(len(X), len(y))

(118985, 118985)

In [78]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [79]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [80]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [97]:
y.value_counts()

1.0    97048
0.0    97048
Name: Charged_Off, dtype: int64

In [90]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

clf = RandomForestClassifier(max_depth = 20)
clf.fit(X_train_scaled, y_train)

train_score = clf.score(X_train_scaled, y_train)
test_score = clf.score(X_test_scaled, y_test)

y_pred = clf.predict(X_test_scaled)
f1_score = f1_score(y_test, y_pred)
roc_score = roc_auc_score(y_test, y_pred)

(train_score, test_score, f1_score, roc_score)



(0.9140219272937103,
 0.8868807188195532,
 0.8740274940903772,
 0.8876156661955668)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

clf = RandomForestClassifier(max_depth = 20)
clf.fit(X_train_scaled, y_train)

train_score = clf.score(X_train_scaled, y_train)
test_score = clf.score(X_test_scaled, y_test)

y_pred = clf.predict(X_test_scaled)
f1_score = f1_score(y_test, y_pred)
roc_score = roc_auc_score(y_test, y_pred)

(train_score, test_score, f1_score, roc_score)

In [91]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import f1_score
# from numpy import mean
# from sklearn.datasets import make_classification
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import RepeatedStratifiedKFold
# from imblearn.pipeline import Pipeline
# from imblearn.over_sampling import SMOTE

# steps = [('over', SMOTE()), ('model', RandomForestClassifier(max_depth=20))]
# pipeline = Pipeline(steps=steps)

# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# scores = cross_val_score(pipeline, X_train_scaled, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

In [92]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

clf = GaussianNB()
clf.fit(X_train_scaled, y_train)

train_score = clf.score(X_train_scaled, y_train)
test_score = clf.score(X_test_scaled, y_test)

y_pred = clf.predict(X_test_scaled)
f1_score = f1_score(y_test, y_pred)
roc_score = roc_auc_score(y_test, y_pred)

(train_score, test_score, f1_score, roc_score)

(0.6169386970021707,
 0.6137581403017064,
 0.4995727865000533,
 0.6153383680274674)

In [93]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import f1_score

# clf = KNeighborsClassifier()
# clf.fit(X_train_scaled, y_train)

# train_score = clf.score(X_train_scaled, y_train)
# test_score = clf.score(X_test_scaled, y_test)

# y_pred = clf.predict(X_test_scaled)
# f1_score = f1_score(y_test, y_pred)

# (train_score, test_score, f1_score)

In [94]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

clf = LogisticRegression()
clf.fit(X_train_scaled, y_train)

train_score = clf.score(X_train_scaled, y_train)
test_score = clf.score(X_test_scaled, y_test)

y_pred = clf.predict(X_test_scaled)
f1_score = f1_score(y_test, y_pred)
roc_score = roc_auc_score(y_test, y_pred)

(train_score, test_score, f1_score, roc_score)

(0.6580592421619542, 0.657406644134861, 0.6694372638695565, 0.6571895300978172)

In [95]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

clf = AdaBoostClassifier()
clf.fit(X_train_scaled, y_train)

train_score = clf.score(X_train_scaled, y_train)
test_score = clf.score(X_test_scaled, y_test)

y_pred = clf.predict(X_test_scaled)
f1_score = f1_score(y_test, y_pred)

roc_score = roc_auc_score(y_test, y_pred)

(train_score, test_score, f1_score, roc_score)


(0.8724067815239195, 0.8749690874618745, 0.862808945571309, 0.8756123805775446)

In [96]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

clf = DecisionTreeClassifier(max_depth = 20)
clf.fit(X_train_scaled, y_train)

train_score = clf.score(X_train_scaled, y_train)
test_score = clf.score(X_test_scaled, y_test)

y_pred = clf.predict(X_test_scaled)
f1_score = f1_score(y_test, y_pred)
roc_score = roc_auc_score(y_test, y_pred)

(train_score, test_score, f1_score, roc_score)


(0.884531365922018, 0.8601104608029017, 0.8449732791303157, 0.8608143161322245)

# GridSearching RandomForest and AdaBoost    

In [99]:
X.shape

(194096, 114)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

param_grid = {
    'max_depth': [5, 20, 50, 100]
    
}

clf = RandomForestClassifier()


train_score = clf.score(X_train_scaled, y_train)
test_score = clf.score(X_test_scaled, y_test)

y_pred = clf.predict(X_test_scaled)
f1_score = f1_score(y_test, y_pred)
roc_score = roc_auc_score(y_test, y_pred)

(train_score, test_score, f1_score, roc_score)